In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
#Reading all the csv to sync and merge
flight = pd.read_csv("C:/Users/sonal/flight-delay-prediction/dataset/flight-combined.csv")
#flight = pd.read_csv("C:/Users/sonal/flight-delay-prediction/dataset/JFK-flights-2015-2024.csv") #AA dataset has a lot of early flights

In [3]:
flight.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410925 entries, 0 to 410924
Data columns (total 17 columns):
 #   Column                                    Non-Null Count   Dtype 
---  ------                                    --------------   ----- 
 0   Carrier Code                              410925 non-null  object
 1   Date (MM/DD/YYYY)                         410925 non-null  object
 2   Flight Number                             410925 non-null  int64 
 3   Tail Number                               410018 non-null  object
 4   Destination Airport                       410925 non-null  object
 5   Scheduled departure time                  410925 non-null  object
 6   Actual departure time                     410925 non-null  object
 7   Scheduled elapsed time (Minutes)          410925 non-null  int64 
 8   Actual elapsed time (Minutes)             410925 non-null  int64 
 9   Departure delay (Minutes)                 410925 non-null  int64 
 10  Wheels-off time                 

In [4]:
#Renaming all columns for ease
flight = flight.rename(columns={
    'Carrier Code' : 'carrier',
    'Date (MM/DD/YYYY)' : 'flight_date',
    'Flight Number' : 'flight_num',
    'Tail Number' : 'tail_num',
    'Destination Airport' : 'dest',
    'Scheduled departure time' : 'scheduled_dep',
    'Actual departure time' : 'actual_dep',
    'Scheduled elapsed time (Minutes)' : 'scheduled_elapsed',
    'Actual elapsed time (Minutes)' : 'actual_elapsed',
    'Departure delay (Minutes)' : 'dep_delay',
    'Wheels-off time' : 'wheels_off_time',
    'Taxi-Out time (Minutes)' : 'taxi_out_time',
    'Delay Carrier (Minutes)' : 'delay_carrier',
    'Delay Weather (Minutes)' : 'delay_weather',
    'Delay National Aviation System (Minutes)' : 'delay_aviation_system',
    'Delay Security (Minutes)' : 'delay_security',
    'Delay Late Aircraft Arrival (Minutes)' : 'delay_late_aircraft'
})

In [5]:
#Checking unique values from multiple columns to check for bad data or incorrect formats before merge
unique_values_dict = {}
for column in flight.columns:
    unique_values_dict[column] = flight[column].unique()

# Print unique values for each column
for column, unique_values in unique_values_dict.items():
    print(f"Unique values in '{column}': {unique_values}")

Unique values in 'carrier': ['AA' 'AS' 'DL' 'HA' 'UA']
Unique values in 'flight_date': ['01-01-2015' '01-01-2016' '01-01-2017' ... '12/31/2021' '12/31/2022'
 '12/31/2023']
Unique values in 'flight_num': [   1    3    9 ...  523 1299 1136]
Unique values in 'tail_num': ['N787AA' 'N798AA' 'N792AA' ... 'N76054' 'N76062' 'N78866']
Unique values in 'dest': ['LAX' 'SFO' 'LAS' 'DFW' 'SJU' 'BOS' 'SAN' 'ORD' 'SEA' 'AUS' 'MCO' 'DCA'
 'MIA' 'STT' 'EGE' 'CLT' 'PHX' 'TUS' 'SNA' 'JAC' 'RDU' 'FLL' 'SAT' 'MSP'
 'ATL' 'PHL' 'DEN' 'PIT' 'PDX' 'PSP' 'SJC' 'HNL' 'TPA' 'SLC' 'PBI' 'MSY'
 'DTW' 'RSW' 'JAX' 'CHS' 'IAH' 'SRQ' 'RIC' 'SAV' 'BUF' 'CVG' 'ANC' 'BZN'
 'BGR' 'ORF' 'CLE' 'BNA' 'ROC' 'SYR']
Unique values in 'scheduled_dep': ['09:00' '12:30' '07:00' '08:00' '11:00' '18:45' '17:29' '16:15' '16:05'
 '07:30' '15:30' '17:20' '14:45' '16:30' '18:00' '10:30' '17:30' '20:45'
 '12:55' '17:10' '18:35' '10:00' '17:00' '19:45' '11:55' '15:45' '08:30'
 '06:52' '06:35' '14:20' '10:55' '14:59' '05:45' '20:20' '19:15'

In [6]:
#Need to convert date into datetime as format change looses data
flight['flight_date'] = pd.to_datetime(flight['flight_date'])

In [7]:
#Standarding format to match with weather dataset
flight['flight_date'] = pd.to_datetime(flight['flight_date'], format='%Y-%m-%d', errors='coerce')

In [8]:
#Verifying if number of records are impacted
flight['flight_date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 410925 entries, 0 to 410924
Series name: flight_date
Non-Null Count   Dtype         
--------------   -----         
410925 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 3.1 MB


In [9]:
#Cross checking for nulls
null_counts = flight.isnull().sum()
null_counts = null_counts[null_counts > 0]  # Filter to only columns with missing values
print(null_counts)
#nulls found in tail-num which is not really important and i might drop it ahead

tail_num    907
dtype: int64


In [10]:
#Checking for duplicates before removing irrelevant columns
print("Number of duplicate rows: ", flight.duplicated().sum())

Number of duplicate rows:  0


In [11]:
#Dropping irrelevant columns
flight = flight.drop('tail_num',axis = 1)

In [12]:
#Creating a combined date-time field to merge weather data with
flight['flight_date'] = pd.to_datetime(flight['flight_date'], format='%Y-%m-%d')
flight['combined_datetime'] = flight['flight_date'].dt.strftime('%Y-%m-%dT') + flight['scheduled_dep']

In [13]:
def round_time(datetime):
    if datetime.minute >= 30:
        rounded_datetime = datetime.replace(minute=0, second=0, microsecond=0) + timedelta(hours=1)
    else:
        rounded_datetime = datetime.replace(minute=0, second=0, microsecond=0)
    if rounded_datetime.hour == 0 and datetime.minute >= 30 and datetime.hour == 23:
        rounded_datetime += timedelta(days=1)
    return rounded_datetime

flight['combined_datetime'] = pd.to_datetime(flight['combined_datetime'])
flight['rounded_datetime'] = flight['combined_datetime'].apply(round_time)

In [14]:
print(flight['combined_datetime'], flight['rounded_datetime'])

0        2015-01-01 09:00:00
1        2015-01-01 12:30:00
2        2015-01-01 07:00:00
3        2015-01-01 08:00:00
4        2015-01-01 11:00:00
                 ...        
410920   2021-12-30 08:00:00
410921   2021-12-30 16:55:00
410922   2021-12-31 08:30:00
410923   2021-12-31 08:00:00
410924   2021-12-31 16:55:00
Name: combined_datetime, Length: 410925, dtype: datetime64[ns] 0        2015-01-01 09:00:00
1        2015-01-01 13:00:00
2        2015-01-01 07:00:00
3        2015-01-01 08:00:00
4        2015-01-01 11:00:00
                 ...        
410920   2021-12-30 08:00:00
410921   2021-12-30 17:00:00
410922   2021-12-31 09:00:00
410923   2021-12-31 08:00:00
410924   2021-12-31 17:00:00
Name: rounded_datetime, Length: 410925, dtype: datetime64[ns]


In [ ]:
flight.to_csv("C:/Users/sonal/flight-delay-prediction/dataset/flight_cleaned.csv")